# Odalys Benitez 

In [1]:
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [10]:
def make_system(beta, gamma, mu, alpha, sigma, delta, pi, tau, rho):
    """
    R= uninfected CD4 lymphocytes
    L=latently infected cells
    E= actively infected cells
    V= free virions
    α	Alpha
    β	Beta
    γ	Gamma
    δ	Delta
    μ	Mu
    π	Pi
    σ,ς *	Sigma
    ρ	Rho
    τ	Tau
    
    returns: System object
    """
    init = State(R=8, L=3, E=2, V=0)
    init /= np.sum(init)

    t0 = 0
    t_end = 7 * 14
    
    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma, alpha=alpha, mu=mu, pi=pi, sigma=sigma, delta=delta, tau=tau, rho=rho)

In [11]:
def update_func(state, t, system):
    """Update the SIR model.
    
    state: State (r, l, e v)
    t: time
    system: System object
    
    returns: State (sir)
    """
    unpack(system)
    
    r, l, e, v = state
    
    #rdelta = (gamma * tau) - (mu * r) - (beta * r * v) 
    #ldelta(rho * rdelta_flow_out_L) - (mu * l) - (alpha * l)
    #edelta = ((1 - rho) * (beta * r * v)) + (alpha * l) - (delta * e)
    #vdelta = (pi * e) - (sigma * v)
    rdelta_flow_in = (gamma * tau)
    rdelta_flow_death = (mu * r)
    rdelta_flow_out_L = (beta * r * v) 
    ldelta_flow_in = (rho * rdelta_flow_out_L) 
    ldelta_flow_death = (mu * l) 
    ldelta_flow_out_E = (alpha * l)
    edelta_flow_in = ((1 - rho) * (rdelta_flow_out_L)) + (ldelta_flow_out_E) 
    edelta_flow_death = (delta * e)
    vdelta_creation = (pi * e) 
    vdelta_death = (sigma * v)
    
    r += rdelta_flow_in - rdelta_flow_death - rdelta_flow_out_L 
    l += ldelta_flow_in - ldelta_flow_death - ldelta_flow_out_E
    e += edelta_flow_in - edelta_flow_death
    v += vdelta_creation - vdelta_death
    
    return State(R=r, L=l, E=e, V=v)